In [2]:
import os
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [86]:
docs = load_pdf_file(data='C:/Users/ASUS/Desktop/Mental Health/data/')


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)

[Document(metadata={'source': 'C:\\Users\\ASUS\\Desktop\\Mental Health\\data\\Australian J Rural Health - 2001 - Fuller - HOW DEFINITION OF MENTAL HEALTH PROBLEMS CAN INFLUENCE HELP SEEKING IN RURAL.pdf', 'page': 0}, page_content='INTRODUCTION\nMental health problems and disorders are a significant\npublic health problem in Australia. The Australian\nNational Mental Health Strategy identifies mental dis-\norders as those that affect a person’s cognitive, emotional\nor social abilities and attract a diagnosis of psychiatric ill-\nness. Mental health problems also affect these abilities,\nbut not to the extent that they warrant a formal illness\ndiagnosis.1 It is estimated that almost one in five Aus-\ntralian children and adolescents suffers from a mental\nhealth problem, while 18% of adults have a mental\ndisorder.2,3\nMcLennan3 reported that only 38% of adult Aus-\ntralians with a mental disorder received help, while Bayer\nand Peay4 reported that of those with a ‘serious’ mental\ndis

In [88]:
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'C:\\Users\\ASUS\\Desktop\\Mental Health\\data\\Australian J Rural Health - 2001 - Fuller - HOW DEFINITION OF MENTAL HEALTH PROBLEMS CAN INFLUENCE HELP SEEKING IN RURAL.pdf', 'page': 0}, page_content='INTRODUCTION\nMental health problems and disorders are a significant\npublic health problem in Australia. The Australian\nNational Mental Health Strategy identifies mental dis-\norders as those that affect a person’s cognitive, emotional\nor social abilities and attract a diagnosis of psychiatric ill-\nness. Mental health problems also affect these abilities,\nbut not to the extent that they warrant a formal illness\ndiagnosis.1 It is estimated that almost one in five Aus-\ntralian children and adolescents suffers from a mental\nhealth problem, while 18% of adults have a mental\ndisorder.2,3\nMcLennan3 reported that only 38% of adult Aus-\ntralians with a mental disorder received help, while Bayer\nand Peay4 reported that of those with a ‘serious’ mental\ndis

In [60]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:

os.environ["LANGCHAIN_TRACING_V2"] = "true"
HF_TOKEN = os.getenv("HF_TOKEN")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

In [ ]:
from langchain_openai import OpenAI


In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
import streamlit as st

llm = Ollama(model="llama3.2-vision")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user queries."),
    ("user", "Question: {question}")
])
chain = LLMChain(prompt=prompt, llm=llm)

In [94]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings
from langchain_community.vectorstores import FAISS


db=FAISS.from_documents(documents[:30],HuggingFaceEmbeddings())

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18496\3229519015.py:8: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  db=FAISS.from_documents(documents[:30],HuggingFaceEmbeddings())


In [112]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriever.invoke("I am suffer from a mental problem. Can you help me?")

In [97]:
retrieved_docs

[Document(metadata={'source': 'C:\\Users\\ASUS\\Desktop\\Mental Health\\data\\Australian J Rural Health - 2001 - Fuller - HOW DEFINITION OF MENTAL HEALTH PROBLEMS CAN INFLUENCE HELP SEEKING IN RURAL.pdf', 'page': 3}, page_content='DEFINITION OF MENTAL HEALTH PROBLEMS: J. FULLER ET AL. 151\nwith fear. According to a social worker, the stigma means\nthat: ‘…a lot of people won’t come in because…mental\nhealth (has) got that stigma…you know, I’m not a nut\ncase…’ and according to a telephone counsellor: ‘…some\nare shocked when you … try to give them a referral to a\nmental health service. Because they may think that’s only\nfor weirdos, people who are mad’. A mental health con-\nsumer and advocate declared similarly that: ‘…because of\nthe stigma attached with mental illness…the last thing\nyou want to do is to go into the system and seek help’.\nThe conventional understanding of mental health\nproblems as implying irremediable insanity leads to a fear\nabout what happens to people who b

In [107]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are a compassionate and knowledgeable mental health assistant. Your role is to understand the user's condition based on their input and provide thoughtful, accurate, and supportive guidance about mental health. Respond to queries with empathy, clarity, and a friendly tone, ensuring the user feels heard, validated, and supported. Avoid making medical diagnoses or prescribing medication, but encourage users to seek professional help when necessary and provide resources or tips to improve mental well-being."
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [114]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [117]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: OPENAI_A**_KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

None
